In [4]:
import kagglehub
import torch
torch.cuda.is_available()

True

In [5]:
weights_dir = kagglehub.model_download(
    "google/gemma/pyTorch/2b-it")

In [6]:
import os
tokenizer_path = os.path.join(weights_dir, "tokenizer.model")

ckpt_path = os.path.join(weights_dir, "gemma-2b-it.ckpt")

In [7]:
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [8]:
import sys

sys.path.append("gemma_pytorch")

In [11]:
from gemma_pytorch.gemma.config import get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

In [12]:
import torch

model_config = get_config_for_2b()
model_config

GemmaConfig(vocab_size=256000, max_position_embeddings=8192, num_hidden_layers=18, num_attention_heads=8, num_key_value_heads=1, hidden_size=2048, intermediate_size=16384, head_dim=256, rms_norm_eps=1e-06, dtype='bfloat16', quant=False, tokenizer='tokenizer/tokenizer.model')

In [13]:
model_config.tokenizer = tokenizer_path

In [14]:
model_config.quant = "quant" in "2b-it"

In [15]:
torch.set_default_dtype(model_config.get_dtype())

In [16]:
device = torch.device("cuda")
device

device(type='cuda')

In [17]:
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)

In [18]:
model = model.to(device)

# INFERENCE

In [19]:
model.generate("Give me a 5 day workout plan, 2 rest days. Targeting chest. Make each workout around 30 mins long",
device = device,
output_len = 100)

'.\n\n**Day 1:**\n\n* Dumbbell Chest Press: 3 sets of 10-12 reps\n* Dumbbell Overhead Press: 3 sets of 10-12 reps\n\n**Day 2:**\n\n* Pull-ups: 3 sets of as many reps as possible\n* Dumbbell Flyes: 3 sets of 10-12 reps\n\n**Day 3:**\n\n* Chest Fly Machine: 3 sets of 1'

In [26]:
def inference(days:int=5, target:str="all around", length:str = "1 hour", rep_range = "8 to 10"):
    prompt = f"Generate me a weekly workout plan. "
    if days:
        prompt += f"Use {days} workout days and {7-days} rest days. "
    if target:
        prompt += f"I want to target the {target}. "
    if length:
        prompt += f"Each workout should be {length} long. "
    if rep_range:
        prompt += f"Aim for the {rep_range} rep range."
    return model.generate(
        prompt,
        device = device,
        output_len = 1000,
    )

In [27]:
print(inference())



**Day 1**

* Warm-up (10 minutes)
* Jumping jacks (8 reps)
* Squats (10 reps)
* Lunges (10 reps)
* Push-ups (8 reps)
* Plank (30 seconds)

**Day 2**

* Warm-up (10 minutes)
* Walking lunge (8 reps)
* Bent-over row (10 reps)
* Glute bridge (10 reps)
* Shoulder press (8 reps)
* Plyo squat (10 reps)

**Day 3**

* Warm-up (10 minutes)
* Deadlifts (10 reps)
* Plank (30 seconds)
* Overhead press (10 reps)
* Chest fly (8 reps)
* Mountain climber (30 seconds)

**Day 4**

* Warm-up (10 minutes)
* Kettlebell swings (10 reps)
* Plank (30 seconds)
* Walking lunge (8 reps)
* Overhead press (10 reps)
* Triceps dip (8 reps)

**Day 5**

* Warm-up (10 minutes)
* Kettlebell goblet squat (10 reps)
* Plank (30 seconds)
* Dead lung (10 reps)
* Hammer curl (10 reps)
* Dumbbell bicep curl (8 reps)

**Day 6**

* Warm-up (10 minutes)
* Cardio (30 minutes)
* Cool-down (10 minutes)

**Day 7**

* Rest
